In [1]:
import os
import sys

In [2]:
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname("__file__"))))))

In [3]:
from tensorboard_utils import variable_summaries

# Model Define

## Perceptron

In [4]:
class Perceptron(object):
    def __init__(self, input_data, output_size):
        self.X = input_data # input_Data == tf.placeholder([Batch,Size])

        # check data type
        if len(input_data.shape) != 2:
            raise ValueError("Wrong input data shape - " + self.__class__.__name__)

        input_size = input_data.shape[-1]
        # block 1
        self.W1 = tf.Variable(tf.zeros([input_size, output_size]))
        self.b1 = tf.Variable(tf.zeros([output_size]))

        self.output = tf.matmul(self.X, self.W1) + self.b1

## Feedforward

In [13]:
class Feedforward(object):
    def __init__(self, input_data, output_size, hidden_layers_size):
        self.X = input_data # input_Data == tf.placeholder([Batch,Size])

        if len(input_data.shape) != 2:
            raise ValueError("Wrong input data shape - " + self.__class__.__name__)
        if type([]) != type(hidden_layers_size):
            raise ValueError("Wrong hidden_layer_size - " + self.__class__.__name__)

        input_size = input_data.shape[-1]
        # blocks
        self.Ws = []
        self.bs = []
        loop = [input_size] + hidden_layers_size + [output_size]
        with tf.name_scope('Feedforward_hidden_layers'):
            for i in range(len(loop) - 1):
                with tf.name_scope('hidden_'+str(i)):
                    with tf.name_scope('weights'):
                        self.Ws.append(tf.Variable(tf.zeros([loop[i], loop[i+1]])))
                        variable_summaries(self.Ws[-1])
                    with tf.name_scope('bias'):
                        self.bs.append(tf.Variable(tf.zeros(loop[i+1])))
                        variable_summaries(self.bs[-1])

        self.output = self.X
        for i in range(len(self.Ws)):
            self.output = tf.matmul(self.output, self.Ws[i]) + self.bs[i]
        tf.summary.histogram('output', self.output)

# Datagen

In [14]:
import numpy as np

In [15]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [16]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
print(X_train.shape, y_train.shape)

(404, 13) (404,)


In [18]:
y_train = y_train.reshape([y_train.shape[0],1])
y_test = y_test.reshape([y_test.shape[0],1])

# Train

In [19]:
import tensorflow as tf

In [20]:
input_layer = tf.placeholder(dtype=tf.float32, shape=[None,13])
y_ = tf.placeholder(tf.float32, [None, 1])
pp = Feedforward(input_layer, 1, [32,16]) # two hidden layer 13->[32->16]->1

In [21]:
loss = tf.reduce_mean(tf.square(pp.output - y_))

optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss)

In [22]:
init = tf.global_variables_initializer()

In [23]:
sess = tf.Session()
sess.run(init)

In [24]:
for step in range(200):
    sess.run(train, feed_dict={input_layer: X_train, y_: y_train})
    if step % 20 == 0:
        print(step, sess.run(loss, feed_dict={input_layer: X_train, y_: y_train}), 
              sess.run(loss, feed_dict={input_layer: X_test, y_: y_test}))

0 406.981 422.036
20 84.6758 83.8429
40 84.6329 83.6169
60 84.6329 83.6148
80 84.6329 83.6148
100 84.6329 83.6148
120 84.6329 83.6148
140 84.6329 83.6148
160 84.6329 83.6148
180 84.6329 83.6148
